# **Machine Learning Applications Final Project Dash**

Date: May 9, 2023

Authors: Óscar Montoya Amores (100451858), Javier Serrano Sánchez-Ocaña (100451673), Miguel Ángel Ponce Martínez (100451309)

#### **LIBRARIES IMPORT**

In [1]:
import joblib

from jupyter_dash import JupyterDash
from dash import dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objs as go
import matplotlib.pyplot as plt


from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import plotly.express as px
import pandas as pd
import numpy as np

# import gensim
import pyLDAvis.gensim as gensimvis
import pyLDAvis

import random

PLOTS A IMPLEMENTAR, BORRAR CUANDO ESTE ACABADO:

-LDA CON LOS TÓPICOS DEPENDIENDO DE CADA GENERO, ES DECIR PODER ELEGIR QUE GENERO PLOTEAR Y QUE SALGAN LAS PALABRAS Q CONFORMAN EL TOPICO

In [2]:
df = pd.read_csv('all_lyrics.csv')
print(df.columns)
print(len(df))

ldag = joblib.load("lda_model.joblib")

from gensim.corpora import Dictionary

dictionary_file = "dictionary.gensim"
D = Dictionary.load(dictionary_file)
from gensim import corpora
# load the corpus from file
corpus = corpora.MmCorpus('corpus.mm')

corpus_words = [[D[word_id] for word_id, count in doc] for doc in corpus]

"""
print(len(corpus))
print(type(ldag))
print(type(corpus))
print(corpus_words[1])
print(type(D))
print("wa" in corpus_words[0])
"""

# Visualize topics using pyLDAvis
vocab = {D[i]: i for i in range(len(D))}
vocab2 = {i: D[i] for i in range(len(D))}
term_frequency = {D[i]: freq for i, freq in D.cfs.items()}
term_frequency = pd.Series(term_frequency)
vis = gensimvis.prepare(ldag, corpus, D, mds='tsne', vocab=vocab2, sort_topics=False, term_frequency=term_frequency)

Index(['song', 'artist', 'type', 'lyrics'], dtype='object')
4000


c:\Users\34649\anaconda3\envs\ML\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
c:\Users\34649\anaconda3\envs\ML\lib\site-packages\pyLDAvis\_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  head(R).drop('saliency', 1)
c:\Users\34649\anaconda3\envs\ML\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\Users\34649\anaconda3\envs\ML\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
c:\Users\34649\anaconda3\envs\ML\lib\site-packages\sklearn\manifold\_t_sne.py:819: FutureWarning: 'square_distances' has been intr

In [3]:
#Open probs df for roc curves, and Y
probs_df = joblib.load("probs_df.joblib")
Y = joblib.load("Y.joblib")
random.seed(33)
encoder = LabelEncoder()
Y = encoder.fit_transform(Y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Y, Y, test_size=0.3, random_state=33)

In [4]:
# Initialize the Dash app
app = JupyterDash(__name__)

server = app.server

# Define the layout of the Dash app
app.layout = html.Div([
    html.H1("Song Genre Topic Modeling"),
    dcc.Dropdown(
        id='dropdown',
        options=[{'label': 'Topic {}'.format(i), 'value': i} for i in range(ldag.num_topics)],
        value=0
    ),
    dcc.Graph(id='topic-type-plot'),
    html.Iframe(
        id='pyldavis',
        srcDoc=pyLDAvis.prepared_data_to_html(vis),
        style={'width': '100%', 'height': '800px'}
    ),
    dbc.Row([dcc.Dropdown(
        id = 'roc_model',
        options=[{'label': i, 'value' : i} for i in list(set(probs_df["MODELS"]))], multi = False, value=list(set(probs_df["MODELS"]))[0]),
    dcc.Dropdown(id = 'roc_classifier',
        options=[{'label': i, 'value' : i} for i in list(set(probs_df["CLASSIFIER"]))], multi = False, value=list(set(probs_df["CLASSIFIER"]))[0])
        ]),
    
    dcc.Graph(id='roc-curves')
])

# Define the update functions for the Dash app
@app.callback(
    Output('pyldavis', 'srcDoc'),
    Input('dropdown', 'value')
)
def update_pyldavis(selected_topic):
    return pyLDAvis.prepared_data_to_html(vis)

@app.callback(
    Output('topic-type-plot', 'figure'),
    Input('dropdown', 'value')
)
def update_topic_type_plot(selected_topic):
    # Get the top 20 words for the selected topic
    top_words = ldag.show_topic(selected_topic, topn=2)
    #print(top_words)
    top_words = [word[0] for word in top_words]
    if "wa" in top_words:
        top_words.remove("wa")
    #print(top_words)
    # Find the indices of the songs in the corpus that contain any of the top 20 words for the selected topic
    indices = []
    #print(len(corpus_words))
    """
    i = 0
    while i<len(corpus_words):
        for word in top_words:
            if word!="wa":
                if word in corpus_words[i]:
                    #print(word)
                    indices.append(i)
                    break
        i+=1
    """

    for i in range(len(corpus_words)):
        j=0
        for word in top_words:
            if word in corpus_words[i]:
                j+=1
        if j==len(top_words):
            indices.append(i)
            
    #print(indices)
    # Filter the dataframe to include only songs with indices in the list of indices
    topic_df = df[df.index.isin(indices)]
    #print(topic_df)
    # Check if any songs match the selected topic
    if topic_df.empty:
        return {'data': [], 'layout': {}}
    # Get the genre of each song in the selected topic
    genres = [genre for genre in topic_df['type']]
    # Count the number of songs in each genre for the selected topic
    genre_counts = pd.Series(genres).value_counts().sort_index()
    # Create the plot
    fig = {
        'data': [{'x': genre_counts.index, 'y': genre_counts.values, 'type': 'bar', 'marker': {'color': ['red', 'green', 'blue', 'orange']}}],
        'layout': {'title': f'Topic {selected_topic} {top_words} Song Genres by LDA' ,'xaxis': {'title': 'Genre'}, 'yaxis': {'title': 'Count'}}
    }
    return fig

@app.callback(
    Output('roc-curves', 'figure'),
    [Input('roc_model', 'value'),
     Input('roc_classifier', 'value')])
def draw_roc(model, classifier):
    model_df = probs_df.index[probs_df['MODELS']==model].tolist()
    classifier_df = probs_df.index[probs_df['CLASSIFIER']==classifier].tolist()
    for i in model_df:
        if i in classifier_df:
            index = i
    selected_roc = probs_df.iloc[index,:]
    prob = selected_roc["PROBS"]

    fpr0, recall0, thresholds0 = metrics.roc_curve(y_test, prob[:,0], pos_label=0) 
    fpr1, recall1, thresholds1 = metrics.roc_curve(y_test, prob[:,1], pos_label=1) 
    fpr2, recall2, thresholds2 = metrics.roc_curve(y_test, prob[:,2], pos_label=2) 
    fpr3, recall3, thresholds3 = metrics.roc_curve(y_test, prob[:,3], pos_label=3) 

    Y_decoded = encoder.inverse_transform([0,1,2,3]) 

    # Create a subplot
    fig, ax = plt.subplots()

    # Plot the ROC curves
    fpr_list = [fpr0, fpr1, fpr2, fpr3]
    recall_list = [recall0, recall1, recall2, recall3]
    color_list = ['red', 'blue', 'green', 'black']
    label_list = [f'ROC curve {Y_decoded[i]}' for i in range(len(Y_decoded))]

    for fpr, recall, color, label in zip(fpr_list, recall_list, color_list, label_list):
        ax.plot(fpr, recall, lw=2.5, label=label, color=color)

    # Convert the plot to a plotly figure
    plotly_fig = go.Figure()
    for trace in fig.get_axes()[0].lines:
        plotly_fig.add_trace(
            go.Scatter(x=trace.get_xdata(), y=trace.get_ydata(), mode='lines', name=trace.get_label(), line=dict(color=trace.get_color()))
    )
    # Add a title to the plot
    plotly_fig.update_layout(title="Different ROC Curves by Genre depending on the Model and Classifier used")
    plotly_fig.update_xaxes(title_text='False Positive Rate')
    plotly_fig.update_yaxes(title_text='True Positive Rate(Recall)')

    return plotly_fig


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)



c:\Users\34649\anaconda3\envs\ML\lib\site-packages\jupyter_dash\jupyter_app.py:310: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  thread.setDaemon(True)


Dash app running on http://127.0.0.1:8050/
